In [39]:
import pandas as pd
from surprise import Reader, Dataset, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy
import numpy as np
from surprise.model_selection import GridSearchCV

In [66]:
ratings = pd.read_csv('/home/vaserkn/VSCode/Anime_rec_sys/ratings.csv')
anime = pd.read_csv('/home/vaserkn/VSCode/Anime_rec_sys/anime.csv')
anime = anime.drop(columns=['Unnamed: 0'])
ratings = ratings.drop(columns=['Unnamed: 0'])

In [79]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=0)

# User-based

## Подбор параметров

In [86]:
params = {'k': [10, 20, 30, 40], 'min_k': [1, 3], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [True]}}

In [87]:
gs = GridSearchCV(KNNWithMeans, param_grid=params, measures=['rmse'])

In [88]:
gs.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [90]:
gs.best_params

{'rmse': {'k': 40,
  'min_k': 3,
  'sim_options': {'name': 'pearson_baseline', 'user_based': True}}}

## Обучение и вычисление rmse

In [91]:
algo = KNNWithMeans(k=40, min_k=3, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.2028


1.2028041603711996

## Рекомендация аниме

In [32]:
def get_rec(user_id, number):
    user_anime_ids = trainset.ur[user_id]
    user_anime_ids = [i[0] for i in user_anime_ids]
    neighbors = algo.get_neighbors(iid=user_id, k=5)
    anime_for_rec = []
    for n in neighbors:
        anime_ids = trainset.ur[n]
        for anime_id in anime_ids:
            if anime_id[0] not in user_anime_ids:
                anime_for_rec.append(anime_id)

    i = 1
    anime_for_rec = sorted(anime_for_rec, key = lambda k: k[1], reverse=True)
    for j in anime_for_rec[:number]:
        try:
            name = anime['Name'].loc[anime['MAL_ID'] == j[0]].values[0]
            print('{}\t{}'.format(i, name))
            i += 1
        except:
            pass


In [33]:
get_rec(333, 10)

1	Coo: Tooi Umi kara Kita Coo
2	Ranma ½: Kessen Tougenkyou! Hanayome wo Torimodose!
3	Shinkyoku Soukai Polyphonica
4	Oruchuban Ebichu
5	Black Cat (TV)
6	Initial D Second Stage
7	Tasogare no Udewa Densetsu
8	Koukaku Kidoutai: Stand Alone Complex 2nd GIG
9	Naruto Movie 3: Dai Koufun! Mikazuki Jima no Animaru Panikku Dattebayo!


# Item-based

## Подбор параметров

In [93]:
params1 = {'k': [40, 50], 'min_k': [1, 3], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [False]}}

In [96]:
gs1 = GridSearchCV(KNNWithMeans, param_grid=params1, measures=['rmse'])

In [97]:
gs1.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [98]:
gs1.best_params

{'rmse': {'k': 40,
  'min_k': 1,
  'sim_options': {'name': 'pearson_baseline', 'user_based': False}}}

## Обучения модели и вычисление rmse

In [99]:
model = KNNWithMeans(k=40, min_k=1, sim_options={'name': 'pearson_baseline', 'user_based': False})
model.fit(trainset)
predictions1 = model.test(testset)
accuracy.rmse(predictions1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.1694


1.1694288878479262

## Рекомендация аниме

In [104]:
def get_recomm(user_id, number):
    anime_id_list = anime['MAL_ID']
    user_anime_id_list = np.unique(ratings.loc[ratings['user_id'] == user_id]['anime_id'].values)

    anime_rating_predictions = []

    for anime_id in anime_id_list:
        if anime_id not in user_anime_id_list:
            anime_rating_predictions.append({'anime_id' : anime_id, 'rating' : model.predict(user_id, anime_id).est})

    i = 1
    for rating in sorted(anime_rating_predictions, key = lambda k: k['rating'], reverse=True)[:number]:
        print('{}\t{}'.format(i, anime['Name'].loc[anime['MAL_ID'] == rating['anime_id']].iloc[0]))
        i += 1 

In [105]:
get_recomm(162, 10)

1	High School! Kimengumi
2	Shinshaku Sengoku Eiyuu Densetsu: Sanada Juu Yuushi
3	City Hunter 2
4	City Hunter: The Secret Service
5	Massugu ni Ikou.
6	Choujikuu Seiki Orguss 02
7	Tsubaki Iro no Purijioone
8	Taiyou no Kiba Dagram
9	Kisaku
10	Gear Fighter Dendoh
